In [4]:
################################################################################
# Import modules
#-------------------------------------------------------------------------------
import numpy as np
import numpy.ma as ma
from astropy.table import Table, QTable
from astropy.io import ascii
import csv


import time


# Import functions from other .py files
from Velocity_Map_Functions import find_phi

from RC_2D_Fit_Functions import Galaxy_Data, \
                                Galaxy_Fitting_iso,\
                                Galaxy_Fitting_NFW, \
                                Galaxy_Fitting_bur, \
                                Hessian_Calculation_Isothermal,\
                                Hessian_Calculation_NFW,\
                                Hessian_Calculation_Burket,\
                                Plotting_Isothermal,\
                                Plotting_NFW,\
                                Plotting_Burkert,\
                                getTidal,\
                                deproject_spaxel,\
                                plot_rot_curve,\
                                plot_diagnostic_panel,\
                                run_MCMC

from Velocity_Map_Functions_cython import rot_incl_iso,\
                                          rot_incl_NFW, \
                                          rot_incl_bur

from mapSmoothness_functions import how_smooth

from os import path

import matplotlib.pyplot as plt
################################################################################

SMOOTHNESS_MORPH_FOLDER = '/Users/richardzhang/Documents/UR_stuff/Research_UR/RotationCurve/2D_RC/main/'

################################################################################
# Importing fitted values & chi2 for each galaxy
fit_mini_iso_name = SMOOTHNESS_MORPH_FOLDER + 'iso_mini_clean2.csv'
fit_mini_iso_table = ascii.read(fit_mini_iso_name,'r')
fit_mini_iso = fit_mini_iso_table[3600:-1]

In [5]:
fit_mini_iso

galaxy_ID,smoothness,Ttype,rho0_b,Rb,SigD,Rd,rho0_h,Rh,incl,phi,x_cen,y_cen,Vsys,chi2
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
9890-9101,1.9723804465455987,1.1827019453048706,-1.6077884411779464,3.4447611506397635,831.7915423525691,5.784087061205006,-2.2985765271657566,4.951829632365486,1.0912196952402293,3.302474372513708,31.53935806209432,32.18248889065099,10.38177933779272,5.413806635210237
9891-12704,1.8428564349894292,4.3866143226623535,-3.980156400504505,5.145653231472333,856.2209440339283,29.99999999979941,-1.5000096001460168,8.863999000365585,0.7964969635653304,2.2726814784831815,39.46034231722298,36.26346891439891,11.60020520654564,7.424425168168707
9891-3702,0.8998782230473965,0.5783001780509949,-1.875700594996021,7.13872008356489,0.10225206717870178,29.999944900605442,-1.7291990745230845,52.13443257557784,0.4441375472292568,1.3024473430057204,14.93735738245399,31.99999233139817,17.042470183245214,31.39045595885857
9891-6102,1.5591825727982955,5.494451522827148,0.3432280567775689,0.6767139354681654,2999.9999444653436,20.438386321094065,1.0721632119414553,0.9261013355344924,0.10397457082296219,6.135953021136265,26.118993376923918,27.17846055239605,6.587982365680259,113.12232403440382
9892-12701,1.527432961474501,4.333785533905029,-0.9136847875520218,1.1906040890299117,1113.31381491404,1.988420852324408,-0.8492282499669356,1.4077952128988809,0.5047682397103948,2.4924338069880174,36.72851974506691,35.74512279363699,-3.89119137932306,6.8087211264801955
9892-12703,0.9041105158612949,6.579679489135742,0.028290934379504477,0.5985265525851652,1015.9745726991902,10.91772220937749,-6.999950279938268,0.1283914094867874,0.46198264582282583,4.229908652339275,36.92212825221853,37.140731178534125,-6.844687536475503,6.018816520011829
9892-12704,1.251399264370438,2.072805881500244,0.9999879882831175,0.3153272475842597,520.2216901984933,26.82121306843072,-2.605186830543703,4.5943817271899965,0.5782080468797138,3.55162584606399,36.279722612402395,36.21968329663946,5.409636039378562,7.084469944647433
9892-12705,1.444861345230394,2.3718879222869873,-0.7061297544270893,0.5634350845473892,2999.9999444653436,29.985871532928147,0.8010655340790906,1.820018003537025,0.12753676846497153,5.703338315446203,39.79509197707809,35.094716654972935,17.666874321619613,27.05798889221454
9892-1902,0.735827880859375,2.777585506439209,-0.5254234289142824,0.829461030448051,1141.2210913596114,10.68431696443926,-1.341848946361888,3.6168335054504435,0.5033143196595176,1.6063762328796924,15.992620520443413,16.99908547614711,4.315419335628142,28.618306838974274


In [6]:
fit_mini_iso = fit_mini_iso_table[360:480]
fit_mini_iso

galaxy_ID,smoothness,Ttype,rho0_b,Rb,SigD,Rd,rho0_h,Rh,incl,phi,x_cen,y_cen,Vsys,chi2
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
11011-3702,1.9344233668606612,0.12078773975372314,-0.11995422790348484,2.8936533410095717,1080.570790045629,12.002732742982822,-1.939978543628158,1.4740397649754489,0.6015772300134241,2.624789751123981,23.967160152031877,22.029828741452995,0.21304531976179897,9.596682153052516
11011-6102,1.4841108194184096,3.2981455326080322,0.6453369480372915,0.4338775561117068,2439.4379536399056,15.53481742878083,-0.3532294602823139,999.9586859793668,0.10014256003468652,5.991333063946502,27.33005692035373,28.63419674909277,17.07801373667808,4.381184086755249
11011-6104,1.1928702300449587,5.439172267913818,-0.9250903546515679,2.457676205113378e-10,972.2070806505552,2.8549806605842742,-2.370126974175826,0.8624450400683434,0.5185568746551845,1.95005715948433,28.462671487609768,26.766102607547577,-10.901415180389801,3.8684227617358395
11012-12701,1.4968822658659406,3.1472842693328857,-0.4894887238665499,4.40852636086291,989.5048400583929,29.99999999196378,1.2985493082621729,0.10006050722680153,0.5513746086951095,1.6976322637682366,37.40200159691346,37.70303888020953,34.87500078000713,28.108088941996623
11012-12702,0.47296493406583917,3.548905611038208,-5.291886215230178,7.204037836202768,2999.9999353840813,18.17754478220228,-1.2947475569665468,1.632373018218745,0.6459449452708573,3.8315478840458392,38.557242450802654,40.660130638926084,-10.179970847678367,75.60714466093185
11012-12705,0.6583112354996347,2.7196872234344482,0.9658628710064814,3.6158847095743343,2999.957387168358,4.015220732300456,-1.0478102068488377,46.88469224258977,0.10154876037291985,0.682092014487242,36.005177847856835,33.433408615039575,-0.21274190379750582,18.14405941549002
11012-3702,1.0473960958501345,1.9786282777786255,-0.04698965141973773,1.702660147558696,2802.628534750837,7.299782483527076,1.055820437406015,0.1343626378180935,0.5212670505556825,1.772365375568518,22.613928625541682,22.01309691803174,-12.665258080147925,115.43184514116476
11012-6101,1.3339108787029623,3.6041605472564697,0.4690371077997786,0.5193071003694403,2325.7526439245876,4.730725024674265,-0.5902826870618321,2.043023863698945,0.32902358993495584,3.9160786018474996,28.2062527732204,27.930043557071407,1.0582098646005393,9.206657860948887
11012-6102,1.538822204109252,3.0250048637390137,-6.999999995066799,0.7142615140815171,1443.5328487331235,1.691800780104983,0.30347455640921933,1.1294192083333867,0.42327159750114757,2.62087414236828,27.975402096238064,27.2389434947399,-26.208649894859917,6.208413129181145


In [7]:
c_iso_MCMC = Table()
c_iso_MCMC['gal_ID'] = fit_mini_iso['galaxy_ID']
c_iso_MCMC['rho0_b'] = np.nan
c_iso_MCMC['Rb'] = np.nan
c_iso_MCMC['SigD'] = np.nan
c_iso_MCMC['Rd'] = np.nan
c_iso_MCMC['rho0_h'] = np.nan
c_iso_MCMC['Rh'] = np.nan
c_iso_MCMC['incl'] = np.nan
c_iso_MCMC['phi'] = np.nan
c_iso_MCMC['x_cen'] = np.nan
c_iso_MCMC['y_cen'] = np.nan
c_iso_MCMC['Vsys'] = np.nan
c_iso_MCMC['chi2'] = np.nan

In [8]:
c_iso_MCMC

gal_ID,rho0_b,Rb,SigD,Rd,rho0_h,Rh,incl,phi,x_cen,y_cen,Vsys,chi2
str11,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
11011-3702,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11011-6102,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11011-6104,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11012-12701,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11012-12702,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11012-12705,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11012-3702,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11012-6101,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
11012-6102,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [14]:
if 2:
    print('ok')

ok


In [13]:
if 0:
    print('ok')